<a href="https://colab.research.google.com/github/cyrilgabriele/AI1/blob/main/Lab3_Task1%2B2_Cyril_Ardi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Monte Carlo with incremental (value iteration) State Value Function**

Monte Carlo with State Value Function: For the first task, you are required to take the Monte Carlo algorithm from the first week's lab (Grid World) and modify it to work with a state-value function instead of a random choice. This means that instead of randomly sampling from all possible actions, you sample according to the estimated state-value function. You should also incorporate a strategy to balance exploration and exploitation using ε-Greedy.

Incremental Monte Carlo: In the second task, extend the Monte Carlo algorithm from the first task to an incremental Monte Carlo method. Incremental Monte Carlo updates the value estimates incrementally, reducing the need to wait until the end of an episode to update values. This should lead to more efficient learning.
Contributors: Cyril Gabriele (gabricyr), Ardi Jasari (jasarard)

In [ ]:
from collections import defaultdict, namedtuple
from enum import Enum
from typing import Tuple, List
import random
from IPython.display import clear_output
import copy
import time
import math

In [ ]:
Point = namedtuple('Point', ['x', 'y'])
class Direction(Enum):
  NORTH = "⬆"
  EAST = "⮕"
  SOUTH = "⬇"
  WEST = "⬅"

  @classmethod
  def values(self):
    return [v for v in self]


In [ ]:
# this is our environment => like self.env = gym.make('CartPole-v0') from Lab02
class SimpleGridWorld(object):

  def __init__(self, width: int = 5, height: int = 5, debug: bool = False):
    print("This is our environment")
    self.width = width
    self.height = height
    self.debug = debug
    self.action_space = [d for d in Direction]
    self.reset()

  def reset(self):
    self.cur_pos = Point(x=0, y=(self.height - 1))
    self.goal = Point(x=(self.width - 1), y=0)
    # If debug, print state
    if self.debug:
      print(self)
    return self.cur_pos, 0, False

  def step(self, action: Direction):
    # Depending on the action, mutate the environment state
    if action == Direction.NORTH:
      self.cur_pos = Point(self.cur_pos.x, self.cur_pos.y + 1)
    elif action == Direction.EAST:
      self.cur_pos = Point(self.cur_pos.x + 1, self.cur_pos.y)
    elif action == Direction.SOUTH:
      self.cur_pos = Point(self.cur_pos.x, self.cur_pos.y - 1)
    elif action == Direction.WEST:
      self.cur_pos = Point(self.cur_pos.x - 1, self.cur_pos.y)
    # Check if out of bounds
    if self.cur_pos.x >= self.width:
      self.cur_pos = Point(self.width - 1, self.cur_pos.y)
    if self.cur_pos.y >= self.height:
      self.cur_pos = Point(self.cur_pos.x, self.height - 1)
    if self.cur_pos.x < 0:
      self.cur_pos = Point(0, self.cur_pos.y)
    if self.cur_pos.y < 0:
      self.cur_pos = Point(self.cur_pos.x, 0)

    # If at goal, terminate
    is_terminal = self.cur_pos == self.goal

    # Constant -1 reward to promote speed-to-goal

    reward = -1

    # If debug, print state
    if self.debug:
      print(self)

    return self.cur_pos, reward, is_terminal

  def peek(self, action: Direction):
  # get next position without mutating the environment
    if action == Direction.NORTH:
      new_pos = Point(self.cur_pos.x, self.cur_pos.y + 1)
    elif action == Direction.EAST:
       new_pos = Point(self.cur_pos.x + 1, self.cur_pos.y)
    elif action == Direction.SOUTH:
       new_pos = Point(self.cur_pos.x, self.cur_pos.y - 1)
    elif action == Direction.WEST:
      new_pos = Point(self.cur_pos.x - 1, self.cur_pos.y)
    # Check if out of bounds
    if new_pos.x >= self.width:
      new_pos = Point(self.width - 1, self.cur_pos.y)
    if new_pos.y >= self.height:
      new_pos = Point(self.cur_pos.x, self.height - 1)
    if new_pos.x < 0:
      new_pos = Point(0, self.cur_pos.y)
    if new_pos.y < 0:
      new_pos = Point(self.cur_pos.x, 0)
    return new_pos

  def get_valid_actions(self):
    # get only the actions that change the current position
    valid_actions = []
    for action in self.action_space:
      if self.peek(action) != self.cur_pos:
        valid_actions.append(action)
    return valid_actions

  def __repr__(self):
    res = ""
    for y in reversed(range(self.height)):
      for x in range(self.width):
        if self.goal.x == x and self.goal.y == y:
          if self.cur_pos.x == x and self.cur_pos.y == y:
            res += "@"
          else:
            res += "o"
          continue
        if self.cur_pos.x == x and self.cur_pos.y == y:
          res += "x"
        else:
          res += "_"
      res += "\n"
    return res

In [ ]:
class MonteCarloGeneration(object):
  def __init__(self, env: object, max_steps: int = 1000, debug: bool = False, decay=50, min_epsilon=0.1):
    self.env = env
    self.max_steps = max_steps
    self.debug = debug
    self.decay = decay
    self.min_epsilon = min_epsilon

  def run(self, n_run) -> List:
    buffer = []
    n_steps = 0 # Keep track of the number of steps so I can bail out if it takes too long
    state, _, _ = self.env.reset() # Reset environment back to start
    terminal = False
    while not terminal: # Run until terminal state

        action = self.choose_action(n_run)

        next_state, reward, terminal = self.env.step(action) # Take action in environment
        buffer.append((state, action, reward)) # Store the result
        state = next_state # Ready for the next step
        n_steps += 1
        if n_steps >= self.max_steps:
          if self.debug:
            print("Terminated early due to large number of steps")
          terminal = True # Bail out if we've been working for too long
    return buffer

    # Self written methods to implement Task 1 from Lab03

  def get_epsilon(self, t):
      """Gets value for epsilon. It declines as we advance in episodes."""
      # Ensures that there's almost at least a min_epsilon chance of randomly exploring
      return max(self.min_epsilon, min(1., 1. - math.log10((t + 1) / self.decay)))

  def choose_action(self, n_run):
    # action_space = all possible actions to perform
    action_space = self.env.action_space

    epsilon = self.get_epsilon(n_run)  # epsilon = [0;1]
    if random.random() < epsilon:
      action = random.choice(action_space)
    else:
      action = max(self.env.action_space, key=lambda a: agent.state_value(self.env.peek(a))) #here choose action which result in the state with the highest value of the state-value function V(s)
    return action


In [ ]:
# This class is our agent
class MonteCarloExperiment(object):
  def __init__(self, generator: MonteCarloGeneration):
    self.generator = generator
    self.values = defaultdict(float)
    self.counts = defaultdict(float)

  def _to_key(self, state):
    return (state)

  def state_value(self, state) -> float:
    key = self._to_key(state)
    if self.counts[key] > 0:
        return self.values[key] / self.counts[key]
    else:
        return 0.0



  def run_episode(self, n_run) -> None:
    trajectory = self.generator.run(n_run) # Generate a trajectory
    #print("this is trjectory: ", trajectory)
    episode_reward = 0
    for i, t in enumerate(reversed(trajectory)): # Starting from the terminal state
      state, action, reward = t
      key = self._to_key(state)
      episode_reward += reward  # Add the reward to the buffer
      self.values[key] += episode_reward # add the cumulated reward until now (= return = V_n-1) to V_n
      self.counts[key] += 1 # Increment counter

In [ ]:
def state_value_2d(env, agent):
    res = ""
    for y in reversed(range(env.height)):
      for x in range(env.width):
        if env.goal.x == x and env.goal.y == y:
          res += "   @  "
        else:
          state_value = agent.state_value(Point(x,y))
          res += f'{state_value:6.2f}'
        res += " | "
      res += "\n"
    return res

In [ ]:
env = SimpleGridWorld() # Instantiate the environment
generator = MonteCarloGeneration(env=env) # Instantiate the trajectory generator
agent = MonteCarloExperiment(generator=generator)
for i in range(1000):
  clear_output(wait=True)
  agent.run_episode(i)
  print(f"Iteration: {i}")
  print(state_value_2d(env, agent))
  # time.sleep(5)

Iteration: 999
-49.61 | -83.70 | -49.54 | -70.05 | -90.27 | 
-42.52 | -71.64 | -61.40 | -63.31 | -80.10 | 
-43.38 | -31.84 | -21.03 | -45.06 | -74.02 | 
-51.38 | -47.71 | -15.25 | -10.33 | -28.23 | 
-64.40 | -46.79 | -29.31 |  -4.74 |    @   | 

